In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 41.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
import pandas as pd
import torch
from datasets import Dataset, load_metric
import random


In [ ]:
def shuffle_df(old_df: pd.DataFrame, cycles: int = 1) -> pd.DataFrame:
  for i in range(cycles):
    new_df = old_df.sample(frac=1).reset_index(drop=True)
  return new_df

In [ ]:
import datasets
import random
from transformers import AutoTokenizer

data = {'text': [], 'class': []}


with open(r'food_requests.txt', 'r') as f:
    for line in f:
        line = line.replace('\n', '')
        line = line.replace('...', ',')
        line = line.lstrip('"').rstrip('"')
        data['text'].append(line)
        data['class'].append('food')


with open(r'not_food_requests.txt', 'r') as f:
    for line in f:
        line = line.replace('\n', '')
        line = line.replace('...', ',')
        line = line.lstrip('"').rstrip('"')
        data['text'].append(line)
        data['class'].append('others')


your_dataset = datasets.Dataset.from_dict(data)


In [ ]:
t = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
template = "This example is {}."

id2labels = ["food", "others"]


def create_input_sequence(sample):
    text = sample["text"]
    label = sample["class"][0]
    contradiction_label = random.choice([x for x in id2labels if x != label])

    encoded_sequence = t(
        text*2, [template.format(label), template.format(contradiction_label)])
    encoded_sequence["labels"] = [1, 0]
    encoded_sequence["input_sentence"] = t.batch_decode(
        encoded_sequence.input_ids)

    return encoded_sequence

In [ ]:
your_dataset = your_dataset.train_test_split(test_size=0.2)
train_ds = your_dataset["train"]
test_ds = your_dataset["test"]

In [ ]:
train_dataset = train_ds.map(
    create_input_sequence, batched=True, batch_size=1, remove_columns=["class", "text"])
test_dataset = test_ds.map(create_input_sequence, batched=True,
                           batch_size=1, remove_columns=["class", "text"])

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments, EvalPrediction
import numpy as np

In [ ]:
from transformers import BartTokenizerFast
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-large-mnli')

In [ ]:
def compute_metrics(p: EvalPrediction):
  metric_acc = load_metric("accuracy")
  metric_f1 = load_metric("f1")
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
  preds = np.argmax(preds, axis = 1)
  result = {}
  result["accuracy"] = metric_acc.compute(predictions = preds, references = p.label_ids)["accuracy"]
  result["f1"] = metric_f1.compute(predictions = preds, references = p.label_ids, average = 'macro')["f1"]
  return result

In [ ]:
model_directory = r'/content'

In [ ]:
model = BartForSequenceClassification.from_pretrained("facebook/bart-large-mnli", num_labels = len(id2labels), ignore_mismatched_sizes = True)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-mnli and are newly initialized because the shapes did not match:
- classification_head.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classification_head.out_proj.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
  output_dir = model_directory,      # Output directory
  num_train_epochs = 32,             # Total number of training epochs
  per_device_train_batch_size = 16,  # Batch size per device during training
  per_device_eval_batch_size = 64,   # Batch size for evaluation
  warmup_steps = 500,                # Number of warmup steps for learning rate scheduler
  weight_decay = 0.01,               # Strength of weight decay
)

trainer = Trainer(
  model = model,                     # The instantiated model to be trained
  args = training_args,              # Training arguments, defined above
  compute_metrics = compute_metrics, # A function to compute the metrics
  train_dataset = train_dataset,     # Training dataset
  eval_dataset = test_dataset,       # Evaluation dataset
  tokenizer = tokenizer              # The tokenizer that was used
)

In [ ]:
trainer.evaluate()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<ipython-input-10-524613e71b68>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_acc = load_metric("accuracy")


{'eval_loss': 0.7369142174720764,
 'eval_accuracy': 0.31896551724137934,
 'eval_f1': 0.30019091256204655,
 'eval_runtime': 5.1017,
 'eval_samples_per_second': 22.738,
 'eval_steps_per_second': 0.392}

In [ ]:
trainer.train()

Step,Training Loss
500,0.040200


TrainOutput(global_step=928, training_loss=0.021665395642199218, metrics={'train_runtime': 427.2408, 'train_samples_per_second': 34.454, 'train_steps_per_second': 2.172, 'total_flos': 776200774251840.0, 'train_loss': 0.021665395642199218, 'epoch': 32.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.6905102029340924e-06,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_runtime': 2.0164,
 'eval_samples_per_second': 57.528,
 'eval_steps_per_second': 0.992,
 'epoch': 32.0}

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification", model = model, tokenizer = tokenizer, device = 0)

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [ ]:
sequences = 'I need food'

In [ ]:
classifier(sequences, id2labels, multi_label=False)

{'sequence': 'I need water',
 'labels': ['water', 'others'],
 'scores': [0.9999951124191284, 4.9359041440766305e-06]}

In [ ]:
model_path = r'/content'

In [ ]:
model.save_pretrained(model_path)

In [ ]:
!zip -r /content/model.zip /content/pytorch_model.bin /content/config.json

	zip warning: name not matched: /content/model/config.json
  adding: content/pytorch_model.bin (deflated 11%)


In [ ]:
from google.colab import files
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
reloadtrainer = BartForSequenceClassification.from_pretrained('/content/')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


In [ ]:
classifier = pipeline("zero-shot-classification", model = reloadtrainer, tokenizer = tokenizer, device = 0)

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [ ]:
classifier(sequences, id2labels, multi_label=False)

{'sequence': 'I need water',
 'labels': ['water', 'others'],
 'scores': [0.9999951124191284, 4.9359041440766305e-06]}